In [1]:
from mlflow import MlflowClient
from pprint import pprint

In [3]:
client = MlflowClient(tracking_uri="")

In [4]:
all_experiments = client.search_experiments()

print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1718822074252, experiment_id='0', last_update_time=1718822074252, lifecycle_stage='active', name='Default', tags={}>]


In [5]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


In [6]:
experiment_description = (
    "A Test Experiment about MNIST Dataset in MLFlow"
)

experiment_tags = {
    "project-name": "Mnist-Lab",
    "mlflow.note.content": experiment_description
}

mnist_experiment = client.create_experiment(
    name="MLFlow-Start",
    tags=experiment_tags
)

In [10]:
# Use search_experiments() to search on the project_name tag key

mnistlab_experiment = client.search_experiments(
    filter_string="tags.`project-name` = 'Mnist-Lab'"
)


In [9]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize


transform = Compose([
    ToTensor(),
    Normalize((0.5,), (0.5,))
])

data_train = MNIST(root="./", download=True, train=True, transform=transform)
data_test = MNIST(root="./", download=True, train=False, transform=transform)

In [19]:

params = {
    "lr": 0.001,
    "epochs": 5
}

In [12]:
from torch.utils.data import DataLoader

train_loaded = DataLoader(data_train, batch_size=32, shuffle=True)
test_loaded = DataLoader(data_test, batch_size=32, shuffle=True)

In [13]:
from torch import nn

class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        self.cnn_layers = nn.Sequential(
            #CONV1
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            #CONV2
            nn.Conv2d(6, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(32*5*5, 200),
            nn.ReLU(),
            nn.Linear(200, 80),
            nn.ReLU(),
            nn.Linear(80, 10),
        )
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x
    

model = MnistModel()

In [14]:
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=params["lr"])
criterion = nn.CrossEntropyLoss()

In [15]:
#Training the model
from tqdm import tqdm
import torch
#Lists to get all the data about training
train_loss, test_loss = [], []
accuracy_train, accuracy_test = [], []
for epoch in range(params["epochs"]):
    total_train_loss = 0
    total_test_loss = 0
    
    model.train()
    
    total = 0
    for index, (image, label) in tqdm(enumerate(train_loaded), desc=f"Fitting Epoch {epoch + 1}"):
        
        optimizer.zero_grad()
        
        pred = model(image)
        
        loss = criterion(pred, label)
        total_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    
    train_accuracy = total / len(data_train)
    total_train_loss = total_train_loss / (index + 1)
    
    accuracy_train.append(train_accuracy)
    train_loss.append(total_train_loss)
    
    #Validating the model
    model.eval()
    total = 0
    for index, (image, label) in tqdm(enumerate(test_loaded), desc="Validating the model"):
        pred = model(image)
        
        loss = criterion(pred, label)
        total_test_loss += loss.item()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    test_accuracy = total / len(data_test)
    total_test_loss = total_test_loss / (index + 1)
    
    accuracy_test.append(test_accuracy)
    test_loss.append(total_test_loss)
    
    print("Epoch: {}/{}  ".format(epoch + 1, params["epochs"]),
            "Training loss: {:.4f}  ".format(total_train_loss),
            "Testing loss: {:.4f}  ".format(total_test_loss),
            "Train accuracy: {:.4f}  ".format(train_accuracy),
            "Test accuracy: {:.4f}  ".format(test_accuracy))

Fitting Epoch 1: 1875it [00:22, 84.65it/s]
Validating the model: 313it [00:03, 94.78it/s] 


Epoch: 1/5   Training loss: 0.2097   Testing loss: 0.0551   Train accuracy: 0.9351   Test accuracy: 0.9829  


Fitting Epoch 2: 1875it [00:26, 70.16it/s]
Validating the model: 313it [00:02, 107.86it/s]


Epoch: 2/5   Training loss: 0.0595   Testing loss: 0.0457   Train accuracy: 0.9818   Test accuracy: 0.9846  


Fitting Epoch 3: 1875it [00:25, 72.57it/s]
Validating the model: 313it [00:02, 109.88it/s]


Epoch: 3/5   Training loss: 0.0410   Testing loss: 0.0366   Train accuracy: 0.9872   Test accuracy: 0.9882  


Fitting Epoch 4: 1875it [00:26, 72.07it/s]
Validating the model: 313it [00:02, 109.41it/s]


Epoch: 4/5   Training loss: 0.0318   Testing loss: 0.0434   Train accuracy: 0.9903   Test accuracy: 0.9864  


Fitting Epoch 5: 1875it [00:25, 73.57it/s]
Validating the model: 313it [00:02, 105.01it/s]

Epoch: 5/5   Training loss: 0.0234   Testing loss: 0.0336   Train accuracy: 0.9922   Test accuracy: 0.9891  


In [16]:
metrics = {
    "accuracy-average-train": sum(accuracy_train) / len(accuracy_train),
    "accuracy-average-test": sum(accuracy_test) / len(accuracy_test),
    "loss-average-train": sum(train_loss) / len(train_loss),
    "loss-average-test": sum(test_loss) / len(test_loss)
}

In [26]:
import mlflow
mlflow.set_tracking_uri("")

experiment = {
    "name": "MLFlow-Start",
    "run-name": "mnist_test",
    "artifact_path": "rf_mnist"
}

mlflow.set_experiment(experiment["name"])

with mlflow.start_run(run_name=experiment["run-name"]) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.pytorch.log_model(
        pytorch_model = model,
        artifact_path = experiment["artifact_path"]
    )
    

2024/06/19 16:55:53 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/06/19 16:55:57 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
